In [2]:
!pip install torch transformers accelerate huggingface_hub[cli] bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.4 MB/s eta 0:00:0000:0100:01


In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # CUDA 장치로 설정
else:
    device = torch.device("cpu")   # CPU 장치로 설정


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "42dot/42dot_LLM-SFT-1.3B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
!nvidia-smi

Sat May 25 12:33:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:05:00.0  On |                  N/A |
|  0%   39C    P2              37W / 170W |   9739MiB / 12288MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=256,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

<||bos||>근육이 커지기 위해서는 충분한 단백질 섭취가 필요합니다.
단백질을 충분히 먹으면 근육을 키우는 데 도움이 됩니다<|endoftext|>


In [7]:
from transformers import pipeline

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer

)
def query(instruction, input=None):
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
    
    print(generator(
        prompt,
        do_sample=True,
        top_p=0.9,
        early_stopping=True,
        max_length=512,
    )[0]['generated_text'])


query("3박 4일 하노이 여행을 가려는데 일정을 짜줘.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<human>: 3박 4일 하노이 여행을 가려는데 일정을 짜줘.

<bot>: 하노이 여행 일정

일정:

Day 1: 하노이 도착 후 호안끼엠(Ho Chi Minh) 시장 방문

Day 2: 호치민 시립 역사 박물관(Bien Hoa Museum) 방문

Day 3: 호치민 시립 역사 박물관(Bien Hoa Museum) 방문

Day 4: 후에(Hue) 또는 롱비엔(Long Bien) 방문

Day 5: 후에(Hue) 또는 롱비엔(Long Bien) 방문

Day 6: 하노이 출발

이 여행 일정은 하노이에서 호안끼엠(Ho Chi Minh) 시장, 호치민 시립 역사 박물관, 후에(Hue) 또는 롱비엔(Long Bien)를 방문하는 일정을 가진다고 되어 있습니다.

하노이에서 호안끼엠(Ho Chi Minh) 시장까지는 지하철, 버스 또는 택시를 이용하면 됩니다. 이 날은 시장 구경하고 맛있는 음식도 맛볼 수 있습니다.

호치민 시립 역사 박물관에는 베트남 역사에 관련된 많은 유물과 작품이 전시되어 있습니다. 하노이에 있는 역사에 관심이 있다면 꼭 들러보세요.

후에(Hue) 또는 롱비엔(Long Bien)를 방문하면, 그 곳의 역사와 문화를 느낄 수 있습니다.

하노이에서 출발하면 일정은 마무리됩니다.


In [8]:
from transformers import TextStreamer, GenerationConfig

streamer = TextStreamer(tokenizer)
def gen(device, instruction, input=''):
    generation_config = GenerationConfig(
        temperature = 0.1,
        max_new_tokens = 512,
        exponential_decay_length_penalty = (512, 1.03),
        eos_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.05,
        do_sample = True,
        top_p = 0.7,
        min_length = 5,
        use_cache = True,
        return_dict_in_generate = True,
    )
    if input:
        prompt = f"<human>: {instruction}\n\n<input>: {input}\n\n<bot>:"
    else:
        prompt = f"<human>: {instruction}\n\n<bot>:"    
        
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to(device),
        generation_config=generation_config,
        streamer=streamer
    )

In [9]:
gen(device, "세종대왕에 대해서 알려줘.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<||bos||><human>: 세종대왕에 대해서 알려줘.

<bot>: 세종대왕은 조선시대의 왕으로, 1418년에 태어나 1450년에 사망한 인물입니다. 그는 한글 창제와 과학 기술 발전에 큰 업적을 남겼습니다.<|endoftext|>
